In [1]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from src.GeoQueryMongo import *
import pandas as pd
import requests
from src.geocode import *

In [2]:
client = MongoClient(f"mongodb://localhost/companies")
db = client.get_database()
data = list(db.companies_selected.find({}))
df=pd.DataFrame(data)

In [3]:
df_airport=df[['lat_a','lng_a','name']]
df_airport.columns=[['latitude','longitude','name']]
df_discos=df[['lat_d','lng_d','name']]
df_discos.columns=[['latitude','longitude','name']]
df_preschool=df[['lat_p','lng_p','name']]
df_preschool.columns=[['latitude','longitude','name']]
df_starbucks=df[['lat_s','lng_s','name']]
df_starbucks.columns=[['latitude','longitude','name']]
df_companies=df[['latitude','longitude','name']]
df_companies.columns=[['latitude','longitude','name']]

objs=[df_companies, df_airport, df_discos, df_preschool, df_starbucks]
df_tableau = pd.concat(objs, axis=0)
df_tableau.reset_index(drop=True, inplace=True)
df_tableau.to_json("./output/df_tableau.json",orient="records")

## Crunch

In [4]:
load_dotenv()
apiKey=os.getenv("crunch")
print("We have apiKey") if apiKey else print("No apiKey")

We have apiKey


In [5]:
url = "https://api.crunchbase.com/v3.1/odm-organizations"

querystring = {"categories":"ux design",
               "organization_types":"company",
               "locations":"Toronto",
               "sort_order":"created_at DESC",
               "page":"1",
               "user_key":apiKey}

response = requests.request("GET", url, params=querystring, json='json')
response

<Response [200]>

In [6]:
res = response.json()
df=pd.json_normalize(res['data']['items'])
df=df[['properties.name','properties.city_name','properties.region_name','properties.country_code']]
df

,properties.name,properties.city_name,properties.region_name,properties.country_code
0,Switch Creative Solutions,Toronto,Ontario,Canada
1,Taskoob,Toronto,Ontario,Canada
2,PATIO Interactive,Toronto,Ontario,Canada
3,Counter Design Studio,Toronto,Ontario,Canada
4,BSI Labs,Toronto,Ontario,Canada
5,Henchmen Studios,Toronto,Ontario,Canada
6,MitreBox User Experience Design,Toronto,Ontario,Canada
7,Akendi,Toronto,Ontario,Canada
8,HumanLogic,Toronto,Ontario,Canada
9,Art & Mechanical,Toronto,Ontario,Canada


In [7]:
geocode('Switch Creative Solutions Toronto')

<Response [200]>


{'type': 'Point', 'coordinates': [-79.34656, 43.66659]}

In [8]:
GeoQueryMongo('Switch Creative Solutions Toronto')

<Response [200]>


,name,category_code,latitude,longitude
0,GPokr,games_video,43.670233,-79.386755
1,Wisevid,games_video,43.670233,-79.386755
